In [ ]:
# CHATBOT AI - CONDIÇÕES GERAIS DAS SEGURADORAS

# LENDO O PDF (CONDIÇÕES GERAIS) E PASSANDO TODO O CONTEÚDO PARA UM TXT
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
chave_openai = "<chave_openai>"
import os
import pandas as pd
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
import openai
from langchain_openai import OpenAIEmbeddings

with open("condicoes_gerais_azul_assinatura.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    
    text = ""
    
    for page in reader.pages:
        text += page.extract_text()
        
with open('condicoes_gerais_azul_assinatura.txt', 'w', encoding="utf-8") as f:
    f.write(text)
    
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

token_ids = tokenizer.encode(text)

def count_tokens(text: str) -> int:
    return len(token_ids)


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

# LENDO O PDF (CONDIÇÕES GERAIS) E PASSANDO TODO O CONTEÚDO PARA UM TXT
with open("condicoes_gerais_azul_assinatura.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    
    text = ""
    
    for page in reader.pages:
        text += page.extract_text()
        
with open('condicoes_gerais_azul_assinatura.txt', 'w', encoding="utf-8") as f:
    f.write(text)
    
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

# Ou, se preferir, imprimir todos os chunks
    
# Verificar quantos tokens há no primeiro chunk
#print(chunks[:2])  # Exibe os primeiros dois chunks

embeddings = OpenAIEmbeddings(openai_api_key=chave_openai, model="text-embedding-ada-002")
decoded_text = tokenizer.decode(token_ids, clean_up_tokenization_spaces=True)

db = FAISS.from_documents(chunks, embeddings)

chat_duvida = str(input("Escreva sua dúvida : "))
query = f"{chat_duvida}"
docs = db.similarity_search(query)
docs[0]

chain = load_qa_chain(OpenAI(openai_api_key=chave_openai, temperature=0), chain_type="stuff")

chain.run(input_documents=docs, question=query)